In [1]:
import sys
# Add the path to the 'src' directory, not the 'src/utils.py' file
sys.path.append('../src')  # This adds the 'src' directory to the sys.path

import numpy as np
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.experimental import enable_halving_search_cv  # Enable HalvingGridSearchCV
from sklearn.model_selection import train_test_split, HalvingGridSearchCV
from sklearn.impute import SimpleImputer

from catboost import CatBoostRegressor

from utils import get_absolute_path, eval_metrics, ColumnsOneHotEncoder

/Users/admin/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


# 1. Dataset

In [3]:
FEATURES = ['percentage_docks_available', 'station_id', 'post_code', 'altitude', 'laboral_day', 'weekday', 'month', 'day', 'hour', 'ctx-4', 'ctx-3', 'ctx-2', 'ctx-1']

TRAIN_PARAMS = {
    'model__iterations': [100, 200],
    'model__depth': [4, 6, 8, 10],
    'model__learning_rate': [0.01, 0.05, 0.1],
    'model__l2_leaf_reg': [1, 3, 4, 5]
}

DATASET = pd.read_csv(get_absolute_path('../data/processed/groupby/stations_final_2023.csv'))

# 2. Pipeline

In [4]:
pipeline = Pipeline([
  ('onehot', ColumnsOneHotEncoder(columns_to_onehot=['station_id', 'weekday', 'month', 'day', 'hour', 'post_code'])),
  ('scaler', StandardScaler()),
  ('imputer', SimpleImputer(strategy='mean')),
  ('model', CatBoostRegressor(verbose=0))
])

# 3. Split

In [5]:
dataset = DATASET[FEATURES]

y = dataset['percentage_docks_available']
X = dataset.drop('percentage_docks_available', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

# 4. Finetunning

In [6]:

# Configure HalvingGridSearchCV
grid_search = HalvingGridSearchCV(pipeline, TRAIN_PARAMS, cv=5, scoring='r2', n_jobs=-1, verbose=2, factor=2)

# Fit the model and find the best parameters
grid_search.fit(X_train, y_train)

# Get the best parameters
best_params = grid_search.best_params_
print(f"Best parameters found: {best_params}")

# Evaluate the best model on the test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Calculate evaluation metrics
rmse, mae, r2 = eval_metrics(y_test, y_pred)

results_df = pd.DataFrame(grid_search.cv_results_)

# Filter relevant columns for the metrics DataFrame
metrics_df = results_df[['params', 'mean_test_score', 'std_test_score', 'mean_train_score', 'std_train_score']].copy()

# Rename columns for clarity
metrics_df.columns = ['Parameters', 'Mean Test R2', 'Std Test R2', 'Mean Train R2', 'Std Train R2']

metrics_df

n_iterations: 6
n_required_iterations: 6
n_possible_iterations: 6
min_resources_: 5138
max_resources_: 164441
aggressive_elimination: False
factor: 2
----------
iter: 0
n_candidates: 54
n_resources: 5138
Fitting 5 folds for each of 54 candidates, totalling 270 fits


/Users/admin/Library/Python/3.9/lib/python/site-packages/sklearn/model_selection/_validation.py:821: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/admin/Library/Python/3.9/lib/python/site-packages/sklearn/model_selection/_validation.py", line 810, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/admin/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_scorer.py", line 266, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
  File "/Users/admin/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_scorer.py", line 353, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "/Users/admin/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_scorer.py", line 86, in _cached_call
    result, _ = _get_response_values(
  File "/Users/admin/Library/Python/3.9/lib/pyth

[CV] END model__depth=4, model__iterations=100, model__l2_leaf_reg=1, model__learning_rate=0.01; total time=   0.6s
[CV] END model__depth=4, model__iterations=100, model__l2_leaf_reg=1, model__learning_rate=0.01; total time=   0.6s
[CV] END model__depth=4, model__iterations=100, model__l2_leaf_reg=1, model__learning_rate=0.05; total time=   0.6s
[CV] END model__depth=4, model__iterations=100, model__l2_leaf_reg=1, model__learning_rate=0.01; total time=   0.6s
[CV] END model__depth=4, model__iterations=100, model__l2_leaf_reg=1, model__learning_rate=0.01; total time=   0.6s
[CV] END model__depth=4, model__iterations=100, model__l2_leaf_reg=1, model__learning_rate=0.01; total time=   0.6s
[CV] END model__depth=4, model__iterations=100, model__l2_leaf_reg=1, model__learning_rate=0.05; total time=   0.6s
[CV] END model__depth=4, model__iterations=100, model__l2_leaf_reg=1, model__learning_rate=0.05; total time=   0.7s
[CV] END model__depth=4, model__iterations=100, model__l2_leaf_reg=1, mo

/Users/admin/Library/Python/3.9/lib/python/site-packages/sklearn/model_selection/_validation.py:821: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/admin/Library/Python/3.9/lib/python/site-packages/sklearn/model_selection/_validation.py", line 810, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/admin/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_scorer.py", line 266, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
  File "/Users/admin/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_scorer.py", line 353, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "/Users/admin/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_scorer.py", line 86, in _cached_call
    result, _ = _get_response_values(
  File "/Users/admin/Library/Python/3.9/lib/pyth

[CV] END model__depth=4, model__iterations=100, model__l2_leaf_reg=1, model__learning_rate=0.1; total time=   0.5s


...

/Users/admin/Library/Python/3.9/lib/python/site-packages/sklearn/model_selection/_search.py:979: UserWarning: One or more of the test scores are non-finite: [       nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan      

[CV] END model__depth=8, model__iterations=200, model__l2_leaf_reg=5, model__learning_rate=0.1; total time=  44.9s
[CV] END model__depth=8, model__iterations=200, model__l2_leaf_reg=5, model__learning_rate=0.05; total time=  53.7s
[CV] END model__depth=8, model__iterations=200, model__l2_leaf_reg=5, model__learning_rate=0.05; total time=  53.7s
[CV] END model__depth=8, model__iterations=200, model__l2_leaf_reg=5, model__learning_rate=0.1; total time=  45.5s
[CV] END model__depth=8, model__iterations=200, model__l2_leaf_reg=5, model__learning_rate=0.1; total time=  53.7s
[CV] END model__depth=8, model__iterations=200, model__l2_leaf_reg=5, model__learning_rate=0.05; total time=  45.8s
[CV] END model__depth=8, model__iterations=200, model__l2_leaf_reg=5, model__learning_rate=0.1; total time=  53.5s
[CV] END model__depth=8, model__iterations=200, model__l2_leaf_reg=5, model__learning_rate=0.05; total time=  53.7s
[CV] END model__depth=8, model__iterations=200, model__l2_leaf_reg=5, model_

/Users/admin/Library/Python/3.9/lib/python/site-packages/sklearn/model_selection/_search.py:979: UserWarning: One or more of the test scores are non-finite: [       nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan      

Best parameters found: {'model__depth': 8, 'model__iterations': 200, 'model__l2_leaf_reg': 5, 'model__learning_rate': 0.1}


,Parameters,Mean Test R2,Std Test R2,Mean Train R2,Std Train R2
0,"{'model__depth': 4, 'model__iterations': 100, ...",NaN,NaN,0.668080,0.005509
1,"{'model__depth': 4, 'model__iterations': 100, ...",NaN,NaN,0.819020,0.006578
2,"{'model__depth': 4, 'model__iterations': 100, ...",NaN,NaN,0.835192,0.006533
3,"{'model__depth': 4, 'model__iterations': 100, ...",NaN,NaN,0.667227,0.005796
4,"{'model__depth': 4, 'model__iterations': 100, ...",NaN,NaN,0.817576,0.007026
...,...,...,...,...,...
103,"{'model__depth': 4, 'model__iterations': 200, ...",0.828791,0.006842,0.833743,0.000946
104,"{'model__depth': 8, 'model__iterations': 200, ...",0.831533,0.006910,0.837289,0.001064
105,"{'model__depth': 8, 'model__iterations': 200, ...",0.835073,0.006776,0.846298,0.000898
106,"{'model__depth': 8, 'model__iterations': 200, ...",0.833160,0.004221,0.835830,0.001045
